In [26]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import sys
sys.path.append('../')
import torchvision
from torchvision.datasets import CIFAR10
from models.network_util import init_net
import torchvision.transforms as transforms
from collections import OrderedDict
from models.network_util import init_weights
from torch.optim import SGD
import numpy as np
# import configs.base_config

In [16]:
t = torch.randn(5, 3)
t1 = t.clone()

In [19]:
t1.requires_grad_(False)

tensor([[-2.7858, -0.9184, -0.1659],
        [-0.2740, -0.6575, -1.2692],
        [ 0.5806, -0.8476, -1.0971],
        [-0.3549,  2.2335,  1.1597],
        [ 1.2895, -0.3405, -0.4631]])

In [24]:
type(t)

torch.Tensor

In [38]:
K = 10
epsilon = 0.001
temp = (K-1)*(1-epsilon)/epsilon
np.log(temp)

9.103979355984773

In [ ]:
9.103979355984773

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from models.resnet_cifar import Resnet50

model = Resnet50()

In [39]:
from models.network_util import LabelSmoothLoss

In [40]:
loss = LabelSmoothLoss(10, 0.1)

In [44]:
input = torch.rand(32, 10)
target = torch.randint(0, 10, (32,))

In [48]:
loss(input, target)

C:\Users\ysy20\Anaconda3\lib\site-packages\torch\nn\functional.py:1906: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


tensor(-0.0994)

In [4]:
model.parameters()

<generator object Module.parameters at 0x000001B8F6E100A0>

In [5]:
bn_params = []
without_bn_params = []
for modu in model.modules():
    print(modu.__class__.__name__)
    classname = modu.__class__.__name__
    if classname.find('BatchNorm') != -1:
        bn_params.append(modu.parameters())
    else:
        without_bn_params.append(modu.parameters())

Resnet50
Sequential
Conv2d
BatchNorm2d
ReLU
Sequential
BuildingBlock
ReLU
Sequential
Conv2d
BatchNorm2d
Sequential
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
BuildingBlock
ReLU
Sequential
Conv2d
BatchNorm2d
Sequential
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
BuildingBlock
ReLU
Sequential
Conv2d
BatchNorm2d
Sequential
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
BuildingBlock
ReLU
Sequential
Conv2d
BatchNorm2d
Sequential
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
BuildingBlock
ReLU
Sequential
Conv2d
BatchNorm2d
Sequential
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
BuildingBlock
ReLU
Sequential
Conv2d
BatchNorm2d
Sequential
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
BuildingBlock
ReLU
Sequential
Conv2d
BatchNorm2d
Sequential
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
BuildingBlock
ReLU
Sequential
Conv2d
BatchNorm2d
Sequential
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
BuildingBlock
ReLU
Sequential
Conv2d
BatchNorm2d
Sequential
Conv2d
BatchNorm2d
ReLU
Conv2d
BatchNorm2d
BuildingBlock
ReLU

In [6]:
bn_params = []
without_bn_params = []
for name, modu in model.named_parameters():
    print(name)

first_layer.first_layer_conv.weight
first_layer.first_layer_conv.bias
first_layer.first_layer_bn.weight
first_layer.first_layer_bn.bias
stages.0.residual.res_conv.weight
stages.0.residual.res_conv.bias
stages.0.residual.res_bn.weight
stages.0.residual.res_bn.bias
stages.0.build_block.conv1.weight
stages.0.build_block.conv1.bias
stages.0.build_block.bn1.weight
stages.0.build_block.bn1.bias
stages.0.build_block.conv2.weight
stages.0.build_block.conv2.bias
stages.0.build_block.bn2.weight
stages.0.build_block.bn2.bias
stages.1.residual.res_conv.weight
stages.1.residual.res_conv.bias
stages.1.residual.res_bn.weight
stages.1.residual.res_bn.bias
stages.1.build_block.conv1.weight
stages.1.build_block.conv1.bias
stages.1.build_block.bn1.weight
stages.1.build_block.bn1.bias
stages.1.build_block.conv2.weight
stages.1.build_block.conv2.bias
stages.1.build_block.bn2.weight
stages.1.build_block.bn2.bias
stages.2.residual.res_conv.weight
stages.2.residual.res_conv.bias
stages.2.residual.res_bn.weigh

In [10]:
skip_list = ['bn']
def add_weight_decay(model, skip_list):
    decay, no_decay = [], []
    for name, param in model.named_parameters():
        
        if not param.requires_grad: 
            continue # frozen weights
        if len(param.shape) == 1 or name.endswith(".bias") or name in skip_list:
            no_decay.append(param)
        else:
            decay.append(param)

    assert len(list(model.parameters())) == (len(decay) + len(no_decay))
        
    return [{'params': no_decay, 'weight_decay': 0.}, {'params': decay}]

add_weight_decay(model, skip_list)

[{'params': [Parameter containing:
   tensor([ 0.0887, -0.1033,  0.1533,  0.0500, -0.0043, -0.0899,  0.0322, -0.1644,
           -0.1269,  0.1159,  0.1328,  0.1373, -0.0589,  0.0236, -0.0066,  0.1283],
          requires_grad=True), Parameter containing:
   tensor([0.4153, 0.2398, 0.4393, 0.8009, 0.3940, 0.2936, 0.8651, 0.8789, 0.1267,
           0.2715, 0.7462, 0.1476, 0.3219, 0.8748, 0.2489, 0.5871],
          requires_grad=True), Parameter containing:
   tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          requires_grad=True), Parameter containing:
   tensor([-0.2313, -0.0511,  0.1358, -0.2304, -0.1213,  0.0458,  0.0356,  0.1058,
            0.1141,  0.0541,  0.1901, -0.1825, -0.0155,  0.0119, -0.0882,  0.2358],
          requires_grad=True), Parameter containing:
   tensor([0.3081, 0.7446, 0.3553, 0.1795, 0.3745, 0.2101, 0.8162, 0.4938, 0.9287,
           0.4836, 0.8829, 0.8631, 0.2543, 0.9668, 0.4171, 0.8764],
          requires_grad=True), Parameter 

In [76]:
decay

[Parameter containing:
 tensor([[[[-1.6546e-01,  8.9877e-02, -1.8506e-01],
           [ 6.3395e-02,  4.6597e-02,  1.6910e-01],
           [ 6.2634e-02, -6.1564e-02,  8.8535e-02]],
 
          [[ 9.1828e-02,  1.4790e-01,  1.4161e-01],
           [ 3.2180e-02,  1.3082e-01,  6.0199e-02],
           [ 1.3347e-01,  1.9156e-01,  1.2475e-01]],
 
          [[ 7.6550e-02, -1.0289e-01,  1.6243e-01],
           [-1.4450e-01,  2.4243e-02,  3.0883e-02],
           [ 1.5157e-01, -4.3159e-02, -1.9497e-02]]],
 
 
         [[[-1.0865e-01,  6.3004e-02, -5.3852e-02],
           [-9.0140e-02,  6.4823e-02,  5.2924e-02],
           [ 9.4336e-02,  1.6512e-01,  1.7153e-01]],
 
          [[-1.2539e-01, -1.2507e-01,  4.0977e-02],
           [-7.5739e-02,  2.0270e-02, -1.6007e-01],
           [ 7.6305e-02,  1.6719e-02,  8.4445e-03]],
 
          [[-7.4961e-02, -8.8285e-02, -9.1432e-02],
           [ 4.9498e-02, -1.2395e-02,  1.6004e-02],
           [ 7.7898e-02,  1.0108e-01,  8.0844e-02]]],
 
 
         [[[ 1.12

In [65]:
bn_params = []
without_bn_params = []
for name, param in model.named_parameters():
    print(name)
    if name.find('bn') != -1:
        bn_params.append(param.data)
    else:
        without_bn_params.append(param.data)

first_layer.0.weight
first_layer.0.bias
first_layer.1.weight
first_layer.1.bias
stages.0.residual.res_conv.weight
stages.0.residual.res_conv.bias
stages.0.residual.res_bn.weight
stages.0.residual.res_bn.bias
stages.0.build_block.conv1.weight
stages.0.build_block.conv1.bias
stages.0.build_block.bn1.weight
stages.0.build_block.bn1.bias
stages.0.build_block.conv2.weight
stages.0.build_block.conv2.bias
stages.0.build_block.bn2.weight
stages.0.build_block.bn2.bias
stages.1.residual.res_conv.weight
stages.1.residual.res_conv.bias
stages.1.residual.res_bn.weight
stages.1.residual.res_bn.bias
stages.1.build_block.conv1.weight
stages.1.build_block.conv1.bias
stages.1.build_block.bn1.weight
stages.1.build_block.bn1.bias
stages.1.build_block.conv2.weight
stages.1.build_block.conv2.bias
stages.1.build_block.bn2.weight
stages.1.build_block.bn2.bias
stages.2.residual.res_conv.weight
stages.2.residual.res_conv.bias
stages.2.residual.res_bn.weight
stages.2.residual.res_bn.bias
stages.2.build_block.con

In [61]:
SGD([
    {'params': bn_params},
    {'params': without_bn_params}
], lr=1e-3)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0

Parameter Group 1
    dampening: 0
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [40]:
p1 = [{'params': bn_params[i], 'weight_decay':0} for i in range(len(bn_params))]

In [41]:
p2 = [{'params': without_bn_params[i]} for i in range(len(without_bn_params))]

In [42]:
p1.extend(p2)

In [44]:
without_bn_params

[<generator object Module.parameters at 0x0000021D739E35C8>,
 <generator object Module.parameters at 0x0000021D739E32B0>,
 <generator object Module.parameters at 0x0000021D739E3CA8>,
 <generator object Module.parameters at 0x0000021D739E3E60>,
 <generator object Module.parameters at 0x0000021D739E3990>,
 <generator object Module.parameters at 0x0000021D739E3F10>,
 <generator object Module.parameters at 0x0000021D739E3888>,
 <generator object Module.parameters at 0x0000021D739E36D0>,
 <generator object Module.parameters at 0x0000021D73A720A0>,
 <generator object Module.parameters at 0x0000021D739E37D8>,
 <generator object Module.parameters at 0x0000021D73A721A8>,
 <generator object Module.parameters at 0x0000021D73A72258>,
 <generator object Module.parameters at 0x0000021D73A722B0>,
 <generator object Module.parameters at 0x0000021D739E3F68>,
 <generator object Module.parameters at 0x0000021D73A72150>,
 <generator object Module.parameters at 0x0000021D73A72360>,
 <generator object Modul

In [43]:
SGD(p2, lr=1e-3)

ValueError: some parameters appear in more than one parameter group

In [24]:
model.stages.

Sequential(
  (0): BuildingBlock(
    (relu): ReLU()
    (residual): Sequential(
      (res_conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
      (res_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (build_block): Sequential(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): BuildingBlock(
    (relu): ReLU()
    (residual): Sequential(
      (res_conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
      (res_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (build_block): Sequential(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), pad

In [15]:
for modu in model.modules():
    if hasattr(modu, 'bn2'):
        print(modu.bn2.weight.data)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [45]:
model.apply(lambda m: print(m))

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU()
Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
)
ReLU()
Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Sequential(
  (res_conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
  (res_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU()
Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Sequential(
  (0): Conv2d(16, 16, kernel_size=(3, 3), s

Resnet50(
  (first_layer): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (stages): Sequential(
    (0): BuildingBlock(
      (relu): ReLU()
      (residual): Sequential(
        (res_conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
        (res_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (build_block): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): BuildingBlock(
      (relu): ReLU()
      (residual): Sequential(
        (res_conv

In [23]:
init_weights(model)

initialize network with normal
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear weight
find batch norm
find conv or linear wei

In [24]:
for module in model.children():
    classname = module.__class__.__name__
    print(classname.find('Conv'))

-1
-1
-1
-1


In [14]:
module.__class__.__name__.find

-1

In [9]:
transformation = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop((32, 32), 4),
    transforms.ToTensor()
])
dataset = CIFAR10('../../dataset/', train=True, transform=transformation, download=True)

Files already downloaded and verified


In [10]:
dataloader = DataLoader(dataset, batch_size=)

In [12]:
init_net(model)

initialize network with normal


Resnet50(
  (first_layer): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (stages): Sequential(
    (0): BuildingBlock(
      (relu): ReLU()
      (residual): Sequential(
        (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (build_block): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): BuildingBlock(
      (relu): ReLU()
      (residual): Sequential(
        (0): Conv2d(16, 16, k

In [26]:
for i, (img, label) in enumerate(dataloader):
    print(img.size())
    if i > 3:
        break

torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
